#### Applied Machine Learning - Mini Project 3 (Tasnim Ahmed, ta1743)

# Decision Trees

In [1]:
import numpy as np 
import sklearn 
import pandas as pd
import math

### Creating the Decision Tree Class

Given the fact both of the datasets (iris and spambase) have continuous features we will implement decision trees that have binary splits.

In [2]:
# Node class - to create node with feature, threshold, branches and label details
class Node: 
    def __init__(self, left = None, right = None, feature = None, threshold = None, label = None):
        self.left = left              # left branch
        self.right = right            # right branch
        self.feature = feature        # on which feature was the node divided upon 
        self.threshold = threshold    # on what threshold of the feature was it divided upon
        
        self.label = label            # place the label if it is a tree node, else none
        
    def check_leaf_node(self):        # check is the node is a leaf (used later for traversal)
        return self.label != None
        
    def label_value(self):        # return the label value if it is a leaf node
        return self.label
            
        
class DecisionTreeClassifier:
    def __init__(self, n_min):
        self.root = None                # the root of the node
        self.n_min = n_min              # the stopping criteria
    
    # function used for fitting the given data 
    def fit(self, X, y):               
        min_sample_split = X.shape[0]*(self.n_min/100)     # set the stopping hyperparameter
        self.root = self.grow_tree(X, y, min_sample_split) # get the root node after growing tree
        
    # recursive function to grow the tree based on threshold and min_sample_split
    def grow_tree(self, X, y, min_sample_split):    
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        # checking the stopping criteria 
        if (n_samples <= min_sample_split or n_labels == 1):
            # find the max label of the dataset and set that as the leaf node value
            leaf_node = Node(label = self.max_label(y.to_numpy()))
            return leaf_node   
           
        # get the best feature and threshold value that splits the given dataset
        split_feature, split_threshold = self.find_best_split(X, y, n_features)
        
        # create child nodes
        X_col = X.iloc[:, split_feature]
        left_ind = np.argwhere(X_col.to_numpy() <= split_threshold).flatten() # get the left branch indices
        right_ind = np.argwhere(X_col.to_numpy() > split_threshold).flatten() # get the right branch indices
        
        # divide the X and y based on the indices for left and right branches 
        X_left, y_left = X.iloc[left_ind], y.iloc[left_ind]     
        X_right, y_right = X.iloc[right_ind], y.iloc[right_ind]
        
        # call the grow tree function again on the left and right branches
        left_node = self.grow_tree(X_left, y_left, min_sample_split)
        right_node = self.grow_tree(X_right, y_right, min_sample_split)
        
        # return the node with feature value, threshold value and its branches 
        return Node(left_node, right_node,  split_feature, split_threshold)
      
    
    # iterates through the columns of the given dataset to find the best split
    def find_best_split(self, X, y, n_features):
        max_ig = 0
        best_feature = None
        best_threshold = None
        
        for i in range(n_features):              # iterate over the columnns 
            X_col = X.iloc[:, i]                 # extract the column with index i
            max_col_ig, max_col_thr = self.col_information_gain(X_col, y)# get the max ig and thresh of column[i]
            
            if max_col_ig > max_ig:  # find the best ig and thresh amongst all the columns of X
                max_ig = max_col_ig
                best_feature = i
                best_threshold = max_col_thr
        
        # return the best feature value and threshold of the column that has highest information gain
        return best_feature, best_threshold
        
    
    # used for labelling the leaf node - finds the most common label amonst the y 
    def max_label(self, y):
        labels = np.unique(y)
        n_labels = len(labels)
        label_dict = {}
        
        # initiated the dictionary pairs {labelX: 0}
        for i in range(n_labels):
            label_dict[labels[i]] = 0
        
        # iterates over the y and counts each label and adds it to the dictionary
        for i in range(n_labels):
            for j in range(len(y)):
                if y[j] == labels[i]:
                    label_dict[labels[i]] += 1
                    
        #find the max label and return that    
        return max(label_dict, key = label_dict.get)
    
    # entropy formula - find the level of disorder of a dataset
    def entropy(self, column):
        counts = np.bincount(column)  # counts the number of labels in the given y column
        probs = counts/len(column)    # the probability of each label in the y column
        entropy = np.sum([p*math.log(p, 2) for p in probs if p > 0])  # calculate the entropy (before negation)
        return -entropy    # negate and return the value
    
    # measure the reduction in entropy of a split
    def information_gain(self, X_col, y, threshold):
        # calculate the original entropy of the target before split
        main_entropy = self.entropy(y)
        
        # creating the branch splits based on the threshold
        splits =  self.split_dataset(X_col, y, threshold)
        branch_splits = [splits[0], splits[1]]
        target_splits = [splits[2], splits[3]]
        
        
        # sum the entropies*prob of the child branches
        sub = 0
        for i in range(len(branch_splits)):
            prob = len(branch_splits[i])/len(X_col)
            sub += prob*self.entropy(target_splits[i])
            
        # subtract from the main_entropy and return the value                            
        return main_entropy - sub
    
    # splits the dataset using the threshold into left and right branches
    def split_dataset(self, X_col, y, threshold):    
        
        left_split, left_target, right_split, right_target = [], [], [], []
        
        #Split the data based on the threshold 
        for i in range(len(X_col)):
            # get the rows less than the threshold - left branch
            if X_col[i] <= threshold:
                left_split.append(X_col[i]) 
                left_target.append(y[i])
            # get the rows more than or equal to the threshold - right branch
            else:
                right_split.append(X_col[i])
                right_target.append(y[i])
    
        return [left_split, right_split, left_target, right_target]
    
    # calculates the information gain for each column 
    def col_information_gain(self, X_col, y):
        X_col = X_col.to_numpy()   # converts to numpy array for indexing
        y = y.to_numpy()           # converts to numpy array for indexing
        
        # dictionary that stores the theshold values with the information gain {threshold: ig}
        col_igs = {}
        max_ig, max_threshold = None, None
        
        for i in range(len(X_col) - 1):
            # calculate the average of adjacent row values
            avg_thr = (X_col[i] + X_col[i+1])/2

            # get the information gain and add it to the dictionary 
            col_igs[avg_thr] = self.information_gain(X_col, y, avg_thr)
            
            # get the max threshold value (the key) from the dictionary
            max_threshold = max(col_igs, key=col_igs.get)
            max_ig = col_igs.get(max_threshold)    # get the value of the max key

        return max_ig, max_threshold
    
    # predicts y value by traversing the tree that was created by the growing tree function 
    def predict_y(self, X):
        y_pred = []
        for i in range(len(X)):
            X_row = X.iloc[i]
            y_pred.append(self.traverse_tree(X_row, self.root))  # append the predicted value to the array
        
        # returns the predicted values as an array
        return y_pred
           
    # traverses the tree recursively until it reaches a leaf node
    def traverse_tree(self, X, node):   
        if node.check_leaf_node():  # if the node is leas node then it stop traversing and returns the label
            return node.label
        
        # if the value is greater then the threshold then it goes to the right branch
        if X.iloc[node.feature] > node.threshold:
            return self.traverse_tree(X, node.right)
        # else go to the left branch
        return self.traverse_tree(X, node.left)        

## Iris Dataset

In [3]:
# Loading the data
iris_data  = pd.read_csv("iris.csv", names = ["f0", "f1", "f2", "f3", "target"])
# Verifying if the data has been loaded properly
iris_data.head()

,f0,f1,f2,f3,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
# Checking the shape of the data
iris_data.shape

(150, 5)

In [5]:
# Separating the features and the target variable columns
X = iris_data.drop('target', axis = 'columns') #independent variables
y = iris_data['target']  #dependent variable

### Relabelling The y Values For Fitting The Data

In [6]:
# set the species values to 0, 1 and 2
y_labelled = []
for i in y:
    if i == "Iris-setosa": 
        y_labelled.append(0)
        
    elif i == "Iris-versicolor":
        y_labelled.append(1)
        
    else:
        y_labelled.append(2)

# convertint the array to panda dataframe
y_labelled = (pd.DataFrame(y_labelled)).iloc[:, 0]

### Fitting The Model and Predicting y Values

In [7]:
from sklearn.metrics import accuracy_score    # importing accuracy for model performance calculation
from sklearn.model_selection import KFold     # importing KFold for splitting the data into training & testing set

In [8]:
# K-Fold Cross Validation
kfold_value = 10   # number of folds used for splitting
kfold = KFold(n_splits = kfold_value)

In [9]:
n_mins = [5, 10, 15, 20]     # early stopping strategy values
                             # used in the DecisionTree Object to calculate the min number of rows need in 
                             # a dataset to stop the tree from growing

n_min_accuracy = []  # average accuracy of the model for each n_min values
all_accs = []        # accuracy of each iteration, within each fold (used for calculating standard deviation)

for n_min in n_mins: # iterate over all the n_min value
    
    fold_accuracy = []  # stores the accuracies of a fold
    for train, test in kfold.split(X):
        # Split the dataset into training and test set
        X_train, X_test = X.iloc[train], X.iloc[test]  
        y_train, y_test = y_labelled.iloc[train], y_labelled.iloc[test]
        
        dt_iris = DecisionTreeClassifier(n_min)  # create the DecisionTree object with a n_min value
        dt_iris.fit(X_train, y_train)            # fit the model with training set
        y_pred = dt_iris.predict_y(X_test)       # predict y-test value with the fitted model
        
        # append the accuracy of each iteration of the folds
        fold_accuracy.append(accuracy_score(y_test, y_pred)) 
        
    # append the fold_accuracies separately for each n_min value
    all_accs.append(fold_accuracy)  
    # calculate the average accuracy of each n_min and append it
    n_min_accuracy.append(round(np.average(fold_accuracy), 2)) 

In [10]:
#standard deviation across the fold
n_min_stds = []
for i in range(len(all_accs)):
    # calculate the standard deviation
    std = np.sqrt(np.sum((all_accs[i] - np.average(all_accs[i]))**2)/len(all_accs[i]))
    # append it to the array
    n_min_stds.append(round(std,3))

### Summarized Performance Of The Model 

In [11]:
# create the table with the average and standard deviation of the accuracies of each n_min value
iris_report = pd.DataFrame(list(zip(n_mins, n_min_accuracy, n_min_stds)), index = None, columns = ["n_min Values", "Average", "Standard Deviation"])
display(iris_report)

print("Overall Accuracy of the Model:", np.average(n_min_accuracy))

,n_min Values,Average,Standard Deviation
0,5,0.95,0.058
1,10,0.93,0.073
2,15,0.93,0.073
3,20,0.93,0.073


Overall Accuracy of the Model: 0.935


## Spambase

In [12]:
# Loading the data
spam_data  = pd.read_csv("spambase.csv", header = None)
spam_data.columns = [*spam_data.columns[:-1], 'target']
# Verifying if the data has been loaded properly
spam_data.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,target
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [13]:
# Checking the shape of the data to make sure it loaded properly
spam_data.shape

(4601, 58)

In [14]:
# Separating the features and the target variable columns
X_spam = spam_data.drop('target', axis = 'columns') #independent variables
y_spam = spam_data['target']  #dependent variable

### Fitting The Model and Predicting y Values

In [15]:
kfold_s = KFold(n_splits = kfold_value)

n_min_spam = []  # average accuracy of the model for each n_min values
all_accs_spam = []      # accuracy of each iteration, within each fold (used for calculating standard deviation)

for n_min in n_mins: # iterate over all the n_min value (used from above)
    
    fold_accuracy = []  # stores the accuracies of a fold
    for train, test in kfold_s.split(X_spam):
        # Split the dataset into training and test set
        X_train, X_test = X_spam.iloc[train], X_spam.iloc[test]  
        y_train, y_test = y_spam.iloc[train], y_spam.iloc[test]
        
        dt_spam = DecisionTreeClassifier(n_min)  # create the DecisionTree object with a n_min value
        dt_spam.fit(X_train, y_train)            # fit the model with training set
        y_pred = dt_spam.predict_y(X_test)       # predict y-test value with the fitted model
        
        # append the accuracy of each iteration of the folds
        fold_accuracy.append(accuracy_score(y_test, y_pred)) 
        
    # append the fold_accuracies separately for each n_min value
    all_accs_spam.append(fold_accuracy)  
    # calculate the average accuracy of each n_min and append it
    n_min_spam.append(round(np.average(fold_accuracy), 2)) 

In [16]:
#standard deviation across the fold
n_min_stds_spam = []
for i in range(len(all_accs_spam)):
    # calculate the standard deviation
    std = np.sqrt(np.sum((all_accs_spam[i] - np.average(all_accs_spam[i]))**2)/len(all_accs_spam[i]))
    # append it to the array
    n_min_stds_spam.append(round(std,3))

### Summarized Performance Of The Model 

In [17]:
spam_report = pd.DataFrame(list(zip(n_mins, n_min_spam, n_min_stds_spam)), index = None, columns = ["n_min Values", "Average", "Standard Deviation"])
display(spam_report)

print("Overall Accuracy of the Model:", np.average(n_min_spam))

,n_min Values,Average,Standard Deviation
0,5,0.88,0.065
1,10,0.88,0.064
2,15,0.84,0.074
3,20,0.83,0.099


Overall Accuracy of the Model: 0.8575
